In [80]:
import torch 
from torch import nn 
from torch.nn.init import xavier_uniform_
import numpy as scinp
from pennylane import numpy as np 

In [81]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using {} device".format(device))

Using cuda:0 device


In [178]:
class Critic(nn.Module):

    def __init__(self, data_dimension, hidden_dimensionality=None):
        super(Critic, self).__init__()

        if hidden_dimensionality is None:
            hidden_dimensionality = [16, 8]

        self.layer_dimensions = [data_dimension] + hidden_dimensionality + [1]

        layers = []

        for i in range(len(self.layer_dimensions) - 1):
            in_dim = self.layer_dimensions[i]
            out_dim = self.layer_dimensions[i + 1]
            linear_layer = nn.Linear(in_dim, out_dim)
            xavier_uniform_(linear_layer.weight)
            layers.append(linear_layer)
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        critic_output = self.network(x)
        return critic_output


In [203]:
j=torch.Tensor([1,2,3,4])
j

tensor([1., 2., 3., 4.])

In [204]:
j.unsqueeze(0)

tensor([[1., 2., 3., 4.]])

In [179]:
critic=Critic(4).to(device)
critic

Critic(
  (network): Sequential(
    (0): Linear(in_features=4, out_features=16, bias=True)
    (1): Linear(in_features=16, out_features=8, bias=True)
    (2): Linear(in_features=8, out_features=1, bias=True)
  )
)

In [188]:
X_minibatch=torch.Tensor(5,4).uniform_(0,1).to(device)
X_minibatch.requires_grad=True
X_minibatch

tensor([[0.2566, 0.0461, 0.6879, 0.5575],
        [0.3824, 0.1080, 0.7619, 0.2831],
        [0.9292, 0.4134, 0.8556, 0.1907],
        [0.5060, 0.2134, 0.8307, 0.6856],
        [0.6327, 0.3293, 0.1791, 0.5815]], device='cuda:0', requires_grad=True)

In [ ]:
0.2566

In [194]:
X_minibatch**2

tensor([[0.0658, 0.0021, 0.4732, 0.3108],
        [0.1463, 0.0117, 0.5805, 0.0801],
        [0.8634, 0.1709, 0.7320, 0.0364],
        [0.2561, 0.0456, 0.6900, 0.4701],
        [0.4003, 0.1084, 0.0321, 0.3381]], device='cuda:0',
       grad_fn=<PowBackward0>)

In [198]:
t=torch.linalg.norm(X_minibatch,ord=2,dim=1)
t

tensor([0.9230, 0.9047, 1.3426, 1.2090, 0.9375], device='cuda:0',
       grad_fn=<CopyBackwards>)

In [201]:
torch.sum(t)/3

tensor(1.7723, device='cuda:0', grad_fn=<DivBackward0>)

In [184]:
for i in range(len(X_minibatch)):
    X_i=X_minibatch[i]
    y_i=critic(X_i)
    print(torch.autograd.grad(outputs=y_i,inputs=X_i)[0].unsqueeze(0))

tensor([[-0.1264,  0.1960, -0.3175,  0.2337]], device='cuda:0')
tensor([[-0.1264,  0.1960, -0.3175,  0.2337]], device='cuda:0')
tensor([[-0.1264,  0.1960, -0.3175,  0.2337]], device='cuda:0')
tensor([[-0.1264,  0.1960, -0.3175,  0.2337]], device='cuda:0')


In [113]:
B=scinp.random.uniform(-1,1,size=(4,4))
B

array([[ 0.91235996, -0.32094347, -0.88413547,  0.74381247],
       [ 0.96290347,  0.02159079,  0.17826059,  0.32000543],
       [-0.19383135, -0.38087511, -0.95024166, -0.19886284],
       [ 0.70306431,  0.41781422,  0.48008937, -0.79094579]])

In [117]:
B[[0,3]]

array([[ 0.91235996, -0.32094347, -0.88413547,  0.74381247],
       [ 0.70306431,  0.41781422,  0.48008937, -0.79094579]])

In [122]:
type(B[np.random.choice(4,size=2)])

numpy.ndarray

In [123]:
def sample_arrays(X, batch_size, device):

    # We sample n=batch_size points from the data
    row_indices = np.random.choice(len(X), size=batch_size)
    X_minibatch = X[row_indices]
    X_minibatch = torch.Tensor(X_minibatch).to(device)

    # We sample n=batch_size points from the generator
    
    row_indices = np.random.choice(len(X), size=batch_size)
    z_minibatch = X[row_indices]
    z_minibatch = torch.Tensor(z_minibatch).to(device)
    
    # We sample n=batch_size noise points
    epsilons = torch.Tensor(batch_size).uniform_(0, 1).to(device)
    return X_minibatch, z_minibatch, epsilons


In [173]:
X_minibatch, z_minibatch, epsilons=sample_arrays(B,2,device)

In [174]:
epsilons=torch.Tensor(np.ones(2)*10).to(device)
epsilons=epsilons[:,np.newaxis]
epsilons

tensor([[10.],
        [10.]], device='cuda:0')

In [175]:
diff=X_minibatch-z_minibatch
diff

tensor([[ 0.0505,  0.3425,  1.0624, -0.4238],
        [ 0.8969,  0.7987,  1.4303, -0.5921]], device='cuda:0')

In [176]:
epsilons*diff

tensor([[ 0.5054,  3.4253, 10.6240, -4.2381],
        [ 8.9690,  7.9869, 14.3033, -5.9208]], device='cuda:0')

In [142]:
X = torch.Tensor([[3, 5],[5, 5],[1, 0]])                                                                                                                                                                          
y = torch.Tensor([7,4])                                                                                                                                                                                   
X*y

tensor([[21., 20.],
        [35., 20.],
        [ 7.,  0.]])